In [4]:
 ### Random forest classifier with pipeline and Hyperparameter tunning 🟠🟢🔵🟣🔴

import seaborn as sns

In [5]:
df = sns.load_dataset('tips')

In [6]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [7]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [8]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [9]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [10]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [11]:
## dependent feature
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [12]:
## convert the categorical feature into the numerical feature

In [13]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [14]:
df['time'] = encoder.fit_transform(df['time'])

In [15]:
df.time.unique()

array([0, 1])

In [16]:
## independent and dependent feature

x = df.drop(labels=['time'], axis =1)
y = df.time

In [17]:
x

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


In [18]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [36]:
## split the data for training and testing the model

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.20, random_state=42)

In [43]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ## handle missing value
from sklearn.preprocessing import StandardScaler  ## feature scaling
from sklearn.preprocessing import OneHotEncoder ## categorical to numerical
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

In [38]:
categorial_cols = ['sex', 'smoker', 'day']
numerical_cols = ['total_bill', 'tip', 'size']

In [39]:
## feature engineering automation
## numerical pipelines
num_pipeline = Pipeline(
steps= [
    ('imputer', SimpleImputer(strategy= 'median')), ## missing value handle
    ('scaler', StandardScaler())   ## feature scaling
]
)

## categorical pipeline
cat_pipeline = Pipeline(
steps= [
    ('imputer', SimpleImputer(strategy= 'most_frequent')), ## missing value handle
    ('onehotencoder', OneHotEncoder())  ## categorical features to numerical
]
)

In [40]:
preprocessor = ColumnTransformer(
    [
       ('num_pipeline', num_pipeline, numerical_cols),
       ('cat_pipeline', cat_pipeline, categorial_cols)
    ]
)

In [41]:
x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

In [61]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [42]:
## model training automation

In [62]:
models = {
    'Random Forest': RandomForestClassifier(),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
    
}

In [51]:
from sklearn.metrics import accuracy_score

In [59]:
def evaluate_model(x_train, y_train, x_test, y_test, models):
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        ## train model
        model.fit(x_train, y_train)
        
        
        ## predict testing data
        y_test_pred = model.predict(x_test)
        
        # get accuracy scores for test data
        
        test_model_score = accuracy_score(y_test, y_test_pred)
        report[list(models.keys())[i]] = test_model_score
        
    return report

In [63]:
evaluate_model(x_train, y_train, x_test, y_test, models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9387755102040817}

In [68]:
classifier = RandomForestClassifier()

In [69]:
## hyperparameter tuning

In [70]:
params = {
    'max_depth': [3,5,10,None],
    'n_estimators' : [100,200,300],
    'criterion': ['gini', 'entropy']
}

In [71]:
from sklearn.model_selection import RandomizedSearchCV

In [72]:
cv= RandomizedSearchCV(classifier, param_distributions=params, scoring='accuracy', cv =5, verbose = 3)

In [73]:
cv.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.974 total time=   0.6s
[CV 2/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.923 total time=   0.7s
[CV 3/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=1.000 total time=   0.7s
[CV 4/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.897 total time=   0.7s
[CV 5/5] END criterion=entropy, max_depth=10, n_estimators=300;, score=0.923 total time=   0.6s
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.974 total time=   0.4s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.923 total time=   0.4s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=1.000 total time=   0.4s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.949 total time=   0.4s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=200;, 

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [74]:
cv.best_params_

{'n_estimators': 200, 'max_depth': None, 'criterion': 'entropy'}

In [75]:
## randomforest regression solve it Internal Assignment